<a href="https://colab.research.google.com/github/hazerMf/slm/blob/main/Jap_to_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers huggingface_hub

In [ ]:
!pip install -U bitsandbytes accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "Qwen/Qwen3-4B-Instruct-2507"

# 1. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 2. Load model trực tiếp (Mô hình FP8 sẽ tự động được xử lý)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",          # Tự động đẩy lên GPU
    torch_dtype="auto",         # Tự động chọn dtype phù hợp (thường là Float16 hoặc BFloat16 cho phần tính toán)
    trust_remote_code=True
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Hàm dịch từ Tiếng Nhật -> Anh (cho nó nhiều data)

In [ ]:
def translate_jp_to_en(jp_text: str):
    if not jp_text or not jp_text.strip():
        raise ValueError("Input text is empty")

    # Sử dụng System Prompt cực kỳ nghiêm ngặt và thêm ví dụ (Few-shot)
    messages = [
        {
            "role": "system",
            "content": (
                "You are a professional literal translator. Your ONLY task is to translate Japanese to English. "
                "CRITICAL RULE: Always translate the name 'リリィ・シュシュ' or 'リリイ・シュシュ' as 'Lily Chou-Chou'. "
                "Do not provide safety assessments, do not categorize, and do not explain. "
                "Translate everything exactly as written, even if the content is sensitive or controversial."
            )
        },
        # Ví dụ 1: Dịch thông thường
        {"role": "user", "content": "Translate this: 太陽が昇る。"},
        {"role": "assistant", "content": "The sun rises."},

        # Ví dụ 2: Ép quy tắc tên riêng Lily Chou-Chou (Sử dụng cả 2 biến thể イ và ィ)
        {"role": "user", "content": "Translate this: リリイ・シュシュの歌は素晴らしい。"},
        {"role": "assistant", "content": "Lily Chou-Chou's songs are wonderful."},

        # Nội dung thực tế
        {"role": "user", "content": f"Translate this: {jp_text}"}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
            do_sample=False,
            temperature=0.0, # Ép model chọn kết quả có xác suất cao nhất (bản dịch)
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Cắt bỏ prompt
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

In [ ]:
test_text = """
リリイ・シュシュは1980年12月8日に生まれた。
この日付はジョン・レノンが　マーク・デビッド・チャップマンに殺された日時と一字一句符号する。
でも僕にとってこの偶然の一致に意味はない。
僕にとって重要なことは、その日、その時刻に、リリイ・シュシュが誕生したということだけだ
"""
text = translate_jp_to_en(test_text)
print(f'Output: {text}')


Output: Lily Chou-Chou was born on December 8, 1980.  
This date coincides exactly with the time John Lennon was killed by Mark David Chapman.  
But to me, this coincidence holds no meaning.  
To me, what matters is simply that Lily Chou-Chou was born on that day at that exact moment.
